<a href="https://colab.research.google.com/github/andrii-tar/pyspark-practice/blob/development/pySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=8a8fe413a787aa0eb0d3dc58acca269f750d1730dfc029fa470dd7363cae5153
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [3]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

In [5]:
spark_session = (SparkSession.builder
                     .master('local')
                     .appName('trainApp')
                     .config(conf=SparkConf())
                     .getOrCreate())